In [ ]:
import numpy as np
import numpy.random as npr
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.datamodule import MSDataModule
from src.model import MSTransformer
from src.constants import MSConstants
C = MSConstants()

In [ ]:
dm = MSDataModule(
    hdf_path='./data/ProteomeTools.hdf',
    batch_size=512,
    train_val_split=0.99,
    cdhit_threshold=0.9,
    cdhit_word_length=5,
    cache_dir='$TMPDIR',
    num_workers=20
)

In [11]:
from src.torch_helpers import start_tensorboard

start_tensorboard('login-2')

In [ ]:
model = MSTransformer(
    residues=C.alphabet,
    ions=C.ions,
    parent_min_charge=C.min_charge,
    parent_max_charge=C.max_charge,
    fragment_min_charge=C.min_frag_charge,
    fragment_max_charge=C.max_frag_charge,
    losses=C.losses,
    model_dim=256,
    model_depth=2,
    num_heads=1,
    lr=5e-4,
    dropout=0.1, 
    max_length=100
)

In [ ]:
from pytorch_lightning import Trainer

!rm -rf ./lightning_logs
trainer = Trainer(
    gpus=1,
    num_sanity_val_steps=0,
    log_every_n_steps=5
)

In [ ]:
trainer.fit(model, dm)

In [ ]:
dm.setup()
for i, batch in enumerate(dm.predict_dataloader()):
    if i == 1:
        break
    
model = model.cpu()
model.eval()
batch['y_pred'] = model.predict_step(batch)

In [ ]:
from src.plotting import faststem
from src.spectrum import fragment_mz_tensor

mz = fragment_mz_tensor(batch['sequence'][0]).ravel()
y = batch['y'][0].detach().cpu().numpy().ravel()
y_pred = batch['y_pred'][0].detach().cpu().numpy().ravel()
y_mask = batch['y_mask'][0].detach().cpu().numpy().ravel()
faststem(mz,y)
faststem(mz,-y_pred)
# faststem(mz,-y_mask*1e5)